## 1. Python dependencies, GCPM model and its compilation

In [ ]:
%%capture

# Mount Google Drive (you can use any storage)
from google.colab import drive
drive.mount('/content/drive')

# Python dependencies
! pip install -q pytecgg spacepy jplephem astropy cartopy

# Manual installation of the plasmaspheric model and executable compilation (this is much easier on Linux)
! git clone https://github.com/mattkjames7/PyGCPM
! cp /content/drive/Shareddrives/Iono_LuGRE/GPCM/data/*.dat PyGCPM/PyGCPM/__data/libgcpm/iri/
! pip install -q -e PyGCPM
%cd /content/PyGCPM/PyGCPM/__data/libgcpm/
! make clean -i > /dev/null
! make > /dev/null
%cd /content/

## 2. Import the necessary packages

In [ ]:
from pathlib import Path
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FuncFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature.nightshade import Nightshade
import numpy as np
import pandas as pd
import polars as pl
from tqdm.notebook import tqdm
from pytecgg.satellites import prepare_ephemeris, satellite_coordinates
from pytecgg.parsing import read_rinex_nav, read_rinex_obs
import jplephem
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, get_body, ITRS
import pyproj
from pyproj import Transformer
from PyGCPM import PyGCPM as PyGCPM
from spacepy import coordinates as coord
from spacepy.time import Ticktock

solar_system_ephemeris.set('jpl');

DATA_PATH = Path('/content/drive/Shareddrives/Iono_LuGRE/GPCM/data/')
CACHE_PATH = Path('/content/drive/Shareddrives/Iono_LuGRE/GPCM/cache_runs/')
PLT_PATH = Path('/content/drive/Shareddrives/Iono_LuGRE/Plots_GFLC/GCPM_Plots')
PLT_UTILS_PATH = Path('/content/drive/Shareddrives/Iono_LuGRE/Plots_GFLC')

## 3. GCPM simulation

Select the `arc_of_interest`

In [ ]:
# Write zero-padded SV names: e.g., E2 --> E02
# Arcs in cronological order for Figure 3:
# 1. E02_OP38_100
# 2. E26_OP38_100
# 3. G24_OP74_100
# 4. G30_OP74_102

arc_of_interest = "E02_OP38_100"


obs_tab = pd.read_csv(Path(PLT_UTILS_PATH, "obs_tab.csv"))
sel = obs_tab.loc[obs_tab["ArcID"] == arc_of_interest].copy()
sel["time"] = pd.to_datetime(sel["time"], errors="raise")
unique_doy = sel["time"].dt.dayofyear.unique()

d = sel['time'].dt.date.values[0]
date_as_int = d.year * 10_000 + d.month * 100 + d.day

times = pl.Series(
    pd.date_range(start=sel["time"].min(), end=sel["time"].max(), freq='30s').to_list()
)

nav_files = [
    Path(
        DATA_PATH,
        f"BRDC00IGS_R_2025{doy:03d}0000_01D_MN.rnx"
    )
    for doy in unique_doy
]

In [ ]:
def parse_arc_id(arc_id: str):
    sv_id = arc_id.split("_")[0]

    if sv_id.startswith("E"):
        constellation = "Galileo"
    elif sv_id.startswith("G"):
        constellation = "GPS"
    else:
        constellation = "unknown"

    return sv_id, constellation

sv_id, constellation = parse_arc_id(arc_of_interest)

In [ ]:
sat_x_all = []
sat_y_all = []
sat_z_all = []

for nav_file in nav_files:
    nav_dict = read_rinex_nav(str(nav_file))
    ephem_dict = prepare_ephemeris(nav_dict, constellation=constellation)

    sat_coords = satellite_coordinates(
        ephem_dict=ephem_dict,
        sv_ids=pl.Series([sv_id] * len(times)),
        gnss_system=constellation,
        epochs=times,
    )

    sat_x_all.append(sat_coords["sat_x"])
    sat_y_all.append(sat_coords["sat_y"])
    sat_z_all.append(sat_coords["sat_z"])

sat_x = np.concatenate(sat_x_all)
sat_y = np.concatenate(sat_y_all)
sat_z = np.concatenate(sat_z_all)

In [ ]:
moon = get_body('moon', Time(times, scale='utc'))
moonECEF = moon.transform_to(ITRS(obstime=times))

In [ ]:
list_of_segments = []
for i in range(len(moonECEF)):
    P1 = [sat_x[i]/1e3, sat_y[i]/1e3, sat_z[i]/1e3]
    P2 = [moonECEF.x.value[i], moonECEF.y.value[i], moonECEF.z.value[i]]
    list_of_segments.append((P1, P2))

In [ ]:
R_Earth = 6.371e3
R1_th = R_Earth         # Inner sphere radius
R2_th = R_Earth + 40e3  # Outer sphere radius
N_steps_mass = 100      # Integration steps for mass calculation

def get_proximity_code(min_distance, R1, R2):
    if min_distance < R1: return 0
    elif min_distance > R2: return 2
    else: return 1

def get_intersection_t_values(P1, V, R):
    a = np.dot(V, V)
    if a < 1e-9: return []
    b = 2 * np.dot(P1, V)
    c = np.dot(P1, P1) - R**2
    discriminant = b**2 - 4 * a * c
    if discriminant < 0: return []
    sqrt_discriminant = np.sqrt(discriminant)
    t1 = (-b + sqrt_discriminant) / (2 * a)
    t2 = (-b - sqrt_discriminant) / (2 * a)
    return [t for t in [t1, t2] if not np.isnan(t)]

def get_segment_colored_subsegments(P1, P2, R1, R2):
    P1 = np.array(P1, dtype=float); P2 = np.array(P2, dtype=float); V = P2 - P1
    t_boundaries = [0.0, 1.0]
    t_boundaries.extend(get_intersection_t_values(P1, V, R1))
    t_boundaries.extend(get_intersection_t_values(P1, V, R2))
    t_boundaries = np.unique([t for t in t_boundaries if 0 <= t <= 1])
    t_boundaries.sort()

    if len(t_boundaries) < 2:
        min_distance = np.linalg.norm(P1)
        code = get_proximity_code(min_distance, R1, R2)
        return [{'start_t': 0.0, 'end_t': 1.0, 'code': code}]

    subsegments = []
    for i in range(len(t_boundaries) - 1):
        t_start = t_boundaries[i]; t_end = t_boundaries[i+1]
        if t_end - t_start < 1e-9: continue
        t_mid = (t_start + t_end) / 2.0
        P_mid = P1 + t_mid * V
        min_distance = np.linalg.norm(P_mid)
        code = get_proximity_code(min_distance, R1, R2)
        subsegments.append({'start_t': t_start, 'end_t': t_end, 'code': code})
    return subsegments

def calculate_segment_mass(P1_list, P2_list, date, ut, subsegments_list, N_steps=N_steps_mass, kp=1):
    P1 = np.array(P1_list, dtype=float); P2 = np.array(P2_list, dtype=float)
    V = P2 - P1
    total_mass = 0.0
    total_lenght=[]
    j=0
    ticktock = Ticktock(['2024-03-03T11:00:00'], 'ISO')
    for sub in subsegments_list:
        if sub['code'] == 1:
            j+=1
            t_start = sub['start_t']; t_end = sub['end_t']
            t_steps = np.linspace(t_start, t_end, N_steps + 1)
            a_min=1e10
            lon_min=1000
            lat_min=1000
            for i in range(N_steps):
                t_mid = (t_steps[i] + t_steps[i+1]) / 2.0
                P_prev = P1 + t_steps[i] * V
                P_curr = P1 + t_steps[i+1] * V
                ds = np.linalg.norm(P_curr - P_prev)

                P_mid_coords = P1 + t_mid * V
                x, y, z = P_mid_coords
                lon, lat, alt = pyproj.transform(ecef, lla, x*1e3, y*1e3, z*1e3)

                cvals = coord.Coords([[x, y, z]], 'GEO', 'car')
                cvals.ticks = ticktock

                sm_coords = cvals.convert('SM', 'car')
                rho = 1
                rho,_,_,_ = PyGCPM.GCPM(x=sm_coords.data[0,0]/R_Earth,
                                      y=sm_coords.data[0,1]/R_Earth,
                                      z=sm_coords.data[0,2]/R_Earth,
                                      Date=date, ut=ut, Kp=kp, Verbose=True)
                a_min = np.min([a_min,alt/1e3])
                if alt/1e3==a_min:
                    lon_min=lon
                    lat_min=lat
                    a_min=alt/1e3
                dM = rho *1e6*1e3* ds
                total_mass += dM
    total_lenght.append([lon_min,lat_min,a_min/1e3])
    return total_mass,total_lenght


def run_segment_analysis(list_of_P1_P2, date, ut):
    analysis_reports = []
    print(f"\t Running PyGCPM Mass Analysis from {R1_th:,.0f} km to {R2_th:,.0f} km, for {date} at {ut} UT")

    for P1_list, P2_list in tqdm(list_of_P1_P2, desc="Calcolo segmenti"):
        report = run_single_segment_analysis(P1_list, P2_list, R1_th, R2_th, date, ut)
        analysis_reports.append(report)
    return analysis_reports

def run_single_segment_analysis(P1_list, P2_list, R1, R2, date, ut):
    P1 = np.array(P1_list, dtype=float)
    P2 = np.array(P2_list, dtype=float)

    subsegments_list = get_segment_colored_subsegments(P1, P2, R1, R2)
    codes_present = set(sub['code'] for sub in subsegments_list)

    final_result = None

    # Rule 1: If ANY code 0 is present (passes inside inner sphere), return None
    if 0 in codes_present:
        final_result = 0.0
        final_lenght= 0.0
    # Rule 2: If ONLY code 2 is present (always outside bigger sphere), return 0.0
    elif codes_present == {2}:
        final_result = 0.0
        final_lenght= 0.0
    # Rule 3: If ANY code 1 is present (in spherical crown), measure the mass
    elif 1 in codes_present:
        mass,lenght = calculate_segment_mass(P1_list, P2_list, date, ut, subsegments_list)
        final_result = mass
        final_lenght= lenght

    return {
        "ResultM": final_result,
        "ResultL": final_lenght,
    }

In [ ]:
path = Path(CACHE_PATH, f"results_{arc_of_interest}.npy")
hour = int(sel['time'].dt.hour.median())

if path.exists():
    results = np.load(path, allow_pickle=True)
else:
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

    results = np.array(
        run_segment_analysis(list_of_segments, date=date_as_int, ut=hour)
    )
    np.save(path, results)

## 4. Plot

In [ ]:
def calculate_tangent_points(df):
    p1 = df[['p1x', 'p1y', 'p1z']].values
    p2 = df[['p2x', 'p2y', 'p2z']].values

    # Direction vector and calculation of u
    v = p2 - p1
    dot_p1_v = np.einsum('ij,ij->i', p1, v)
    dot_v_v  = np.einsum('ij,ij->i', v, v)
    u = - dot_p1_v / dot_v_v

    # Tangent Point (TP) coordinates
    tp = p1 + v * u[:, np.newaxis]

    # TP height
    tp_dist_center = np.linalg.norm(tp, axis=1)
    tp_height = tp_dist_center - 6371.0

    return tp, tp_height

tp_coords, tp_heights = calculate_tangent_points(sel)

sel['tp_x'] = tp_coords[:, 0]
sel['tp_y'] = tp_coords[:, 1]
sel['tp_z'] = tp_coords[:, 2]
sel['tp_h'] = tp_heights

# LT and UT
sel['lt_ip'] = sel['lt_ip'].str.slice(0, 5)
sel['ut_time'] = sel['time'].astype(str).str.slice(11, 16)

In [ ]:
def get_lat_lon(x, y, z):
    lon = np.degrees(np.arctan2(y, x))
    hyp = np.sqrt(x**2 + y**2)
    lat = np.degrees(np.arctan2(z, hyp))
    return lat, lon

# def plot_integrated_results(results, times, sel, title='', font_scale=1.0, save=False):
#     N_GEODETICS = 18
#     LABEL_EVERY = 2
#     ALPHA_GRID = 0.4

#     # --- 1. Preparazione Dati TEC ---
#     segment_masses = [report.get("ResultM", np.nan) for report in results]
#     model_tec = np.array(segment_masses).flatten() / 1e16
#     gflc = sel["gflc"].to_numpy()

#     # Allineamento all'ultimo punto
#     shift = model_tec[-1] - gflc[-1] if len(model_tec) > 0 else 0.0
#     gflc_shifted = gflc + shift

#     # --- 2. Setup Figura (2 Panel) ---
#     fig = plt.figure(figsize=(12, 12))
#     gs = GridSpec(2, 2, width_ratios=[1, 0.02], height_ratios=[0.8, 1.2], hspace=0.0, wspace=0.25)

#     ax0 = fig.add_subplot(gs[0, 0])
#     ax_map = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())

#     cax1 = fig.add_subplot(gs[0, 1])
#     cax_map = fig.add_subplot(gs[1, 1])

#     # --- PANEL 0: TEC (Primary Y) ---
#     ax0.plot(times, model_tec, "k", lw=2, label="Modeled (GCPM)", zorder=10)
#     ax0.plot(sel["time"], gflc_shifted, "tab:red", lw=2, label="Measured (LuGRE)", zorder=10)
#     ax0.set_ylabel("TEC [TECu]", fontsize=10 * font_scale)

#     # Legenda al centro a sinistra
#     ax0.legend(frameon=False, loc='center left')
#     ax0.grid(True, ls="--", alpha=ALPHA_GRID, zorder=0)

#     # --- PANEL 0: ALTITUDE (Secondary Y - Twin) ---
#     ax0_twin = ax0.twinx()
#     sc_alt = ax0_twin.scatter(sel["time"], sel["alt_ip"], c=sel["lat_ip"], cmap="YlGnBu", s=10, label='Altitude', zorder=1, alpha=0.9)
#     ax0_twin.set_yscale('log')
#     ax0_twin.set_ylabel("Altitude [km]", fontsize=10 * font_scale)

#     # Colorbar per l'altitudine
#     cb1 = fig.colorbar(sc_alt, cax=cax1)
#     cb1.set_label("Latitude", fontsize=9*font_scale)
#     cb1.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x:.0f}°"))

#     # Formattazione assi temporali
#     date_str = sel['time'].dt.date.values[0].strftime('%-d %B %Y')
#     ax0.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
#     ax0.set_xlabel(date_str, fontsize=10*font_scale, labelpad=10)

#     # --- PANEL 1: CARTINA GEO ---
#     terminator_date = pd.to_datetime(sel['time'].iloc[len(sel)//2])
#     ax_map.set_global()
#     ax_map.add_feature(cfeature.LAND, facecolor='#f5f5f5')
#     ax_map.add_feature(cfeature.OCEAN, facecolor='#cceeff')
#     ax_map.add_feature(cfeature.COASTLINE, linewidth=0.6)
#     ax_map.add_feature(Nightshade(terminator_date, alpha=0.15))

#     gl = ax_map.gridlines(draw_labels=True, lw=0.5, color='gray', alpha=ALPHA_GRID, ls='--')
#     gl.top_labels = gl.right_labels = False

#     indices = np.linspace(0, len(sel) - 1, N_GEODETICS, dtype=int)
#     subset = sel.iloc[indices]

#     colors_map = plt.get_cmap('inferno')(np.linspace(0, 0.90, 256))
#     cmap_map = mcolors.LinearSegmentedColormap.from_list('inferno_cut', colors_map)
#     norm_map = plt.Normalize(vmin=subset['tp_h'].min(), vmax=subset['tp_h'].max())

#     R_terra = 6371.0
#     for i, (index, row) in enumerate(subset.iterrows()):
#         p1 = np.array([row['p1x'], row['p1y'], row['p1z']])
#         p2 = np.array([row['p2x'], row['p2y'], row['p2z']])
#         v = p2 - p1

#         u_vals = np.linspace(0, 1, 50)
#         pts = p1 + u_vals[:, np.newaxis] * v
#         lats, lons = get_lat_lon(pts[:,0], pts[:,1], pts[:,2])
#         h_vals = np.linalg.norm(pts, axis=1) - R_terra

#         for j in range(len(lons)-1):
#             ax_map.plot([lons[j], lons[j+1]], [lats[j], lats[j+1]],
#                         color=cmap_map(norm_map((h_vals[j] + h_vals[j+1])/2)),
#                         linewidth=1.5, transform=ccrs.Geodetic())

#         if i % LABEL_EVERY == 0:
#             lt_s = pd.to_datetime(row['lt_ip']).strftime('%H:%M')
#             ut_s = pd.to_datetime(row['ut_time']).strftime('%H:%M')
#             ax_map.text(lons[0], lats[0], lt_s, transform=ccrs.PlateCarree(), fontsize=7*font_scale, ha='right', va='bottom')
#             ax_map.text(lons[-1], lats[-1], ut_s, transform=ccrs.PlateCarree(), fontsize=7*font_scale, ha='left', va='bottom')

#         u_tp = - np.dot(p1, v) / np.dot(v, v)
#         if 0 <= u_tp <= 1:
#             ptp = p1 + u_tp * v
#             tplat, tplon = get_lat_lon(ptp[0], ptp[1], ptp[2])
#             ax_map.plot(tplon, tplat, 'ko', ms=3, transform=ccrs.Geodetic())

#     # Regolazione fine posizione colorbar per evitare sovrapposizioni
#     pos_map = cax_map.get_position()
#     new_h_map = pos_map.height * 0.73
#     new_y0_map = pos_map.y0 + (pos_map.height - new_h_map) / 2
#     cax_map.set_position([pos_map.x0, new_y0_map, pos_map.width, new_h_map])

#     for ax in (ax0, ax0_twin):
#         for orient_ in ['top', 'bottom', 'left', 'right']:
#           ax.spines[orient_].set_linewidth(0.5)
#         ax.tick_params(axis='both', labelsize=9*font_scale)

#     sm = plt.cm.ScalarMappable(cmap=cmap_map, norm=norm_map)
#     cb_map = fig.colorbar(sm, cax=cax_map)
#     cb_map.set_label('Local height [km]', fontsize=9*font_scale)

#     if save:
#         plt.savefig(Path(PLT_PATH, f"{title}.png"), dpi=400, bbox_inches='tight')

#     plt.show()

In [ ]:
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Computer Modern Serif", "DejaVu Serif"],
})

def plot_integrated_results(results, times, sel, title='', left_letter='', right_letter='',
                            font_scale=1.0, save=False, show_colorbar=True, n_geodetics=15,
                            label_every=2, legend_loc='best'):
    ALPHA_GRID = 0.4

    # --- 1. Preparation of TEC Data ---
    segment_masses = [report.get("ResultM", np.nan) for report in results]
    model_tec = np.array(segment_masses).flatten() / 1e16
    gflc = sel["gflc"].to_numpy()
    shift = model_tec.min() - gflc.min() if len(model_tec) > 0 else 0.0
    gflc_shifted = gflc + shift

    # --- 2. Fig. Setup ---
    fig = plt.figure(figsize=(20, 9))
    gs = GridSpec(2, 2, height_ratios=[1, 0.05], width_ratios=[1, 1.2], hspace=0.2, wspace=0.23)

    ax0 = fig.add_subplot(gs[0, 0])
    ax_map = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree())

    if show_colorbar:
        cax1 = fig.add_subplot(gs[1, 0])
        cax_map = fig.add_subplot(gs[1, 1])

    # --- LEFT PANEL: TEC & ALTITUDE ---
    sv_id = title.split("_")[0]
    ax0.plot(times, model_tec, "k", lw=2, label="Modelled", zorder=10)
    ax0.plot(sel["time"], gflc_shifted, "tab:red", lw=2, label="Measured", zorder=10)
    ax0.set_ylabel("TEC [TECu]", fontsize=10 * font_scale)
    ax0.legend(frameon=False, loc=legend_loc, title=f'{sv_id}', fontsize=8*font_scale, title_fontsize=8*font_scale)
    ax0.grid(True, ls="--", alpha=ALPHA_GRID, zorder=0)

    ax0_twin = ax0.twinx()
    norm_lat = plt.Normalize(vmin=-90, vmax=10)
    sc_alt = ax0_twin.scatter(sel["time"], sel["alt_ip"], c=sel["lat_ip"],
                              cmap="YlGnBu", s=10, zorder=1, alpha=0.9, norm=norm_lat)

    ax0_twin.set_yscale('log')
    ax0_twin.set_ylabel("Altitude [km]", fontsize=10 * font_scale)

    if show_colorbar:
        cb1 = fig.colorbar(sc_alt, cax=cax1, orientation='horizontal', ticks=np.arange(-90, 11, 20))
        cb1.set_label("Latitude", fontsize=9*font_scale)
        cb1.ax.xaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x:.0f}°"))
        cb1.ax.tick_params(labelsize=8*font_scale)

    date_str = sel['time'].dt.date.values[0].strftime('%-d %B %Y')
    ax0.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    ax0.set_xlabel(date_str, fontsize=10*font_scale)

    # --- RIGHT PANEL: GEO ---
    terminator_date = pd.to_datetime(sel['time'].iloc[len(sel)//2])
    ax_map.set_global()
    ax_map.add_feature(cfeature.LAND, facecolor='#f5f5f5')
    ax_map.add_feature(cfeature.OCEAN, facecolor='#cceeff')
    ax_map.add_feature(cfeature.COASTLINE, linewidth=0.6)
    ax_map.add_feature(Nightshade(terminator_date, alpha=0.15))

    gl = ax_map.gridlines(draw_labels=True, lw=0.5, color='gray', alpha=ALPHA_GRID, ls='--')
    gl.top_labels = gl.right_labels = False
    gl.xlabel_style = {'size': 9 * font_scale}
    gl.ylabel_style = {'size': 9 * font_scale}

    colors_map = plt.get_cmap('inferno')(np.linspace(0, 0.90, 256))
    cmap_map = mcolors.LinearSegmentedColormap.from_list('inferno_cut', colors_map)
    norm_h = plt.Normalize(vmin=200, vmax=3700)

    indices = np.linspace(0, len(sel) - 1, n_geodetics, dtype=int)
    subset = sel.iloc[indices]
    R_terra = 6371.0
    for i, (index, row) in enumerate(subset.iterrows()):
        p1 = np.array([row['p1x'], row['p1y'], row['p1z']])
        p2 = np.array([row['p2x'], row['p2y'], row['p2z']])
        v = p2 - p1
        u_vals = np.linspace(0, 1, 50)
        pts = p1 + u_vals[:, np.newaxis] * v
        lats, lons = get_lat_lon(pts[:,0], pts[:,1], pts[:,2])
        h_vals = np.linalg.norm(pts, axis=1) - R_terra
        for j in range(len(lons)-1):
            ax_map.plot([lons[j], lons[j+1]], [lats[j], lats[j+1]],
                        color=cmap_map(norm_h((h_vals[j] + h_vals[j+1])/2)),
                        linewidth=1.5, transform=ccrs.Geodetic())

        if i % label_every == 0:
            lt_s = pd.to_datetime(row['lt_ip']).strftime('%H:%M')
            ut_s = pd.to_datetime(row['ut_time']).strftime('%H:%M')
            ax_map.text(lons[0], lats[0], lt_s, transform=ccrs.PlateCarree(), fontsize=7*font_scale, ha='right', va='bottom')
            ax_map.text(lons[-1], lats[-1], ut_s, transform=ccrs.PlateCarree(), fontsize=7*font_scale, ha='left', va='bottom')

        u_tp = - np.dot(p1, v) / np.dot(v, v)
        if 0 <= u_tp <= 1:
            ptp = p1 + u_tp * v
            tplat, tplon = get_lat_lon(ptp[0], ptp[1], ptp[2])
            ax_map.plot(tplon, tplat, 'ko', ms=3.25, transform=ccrs.Geodetic())

    fig.canvas.draw()
    map_pos = ax_map.get_position()
    ax0_pos = ax0.get_position()
    ax0.set_position([ax0_pos.x0, map_pos.y0, ax0_pos.width, map_pos.height])

    fig.text(ax0_pos.x1 - 0.008, map_pos.y0 + 0.05, f"{left_letter})",
             fontsize=11*font_scale, ha='right', va='bottom', fontweight='bold')
    fig.text(map_pos.x1 - 0.008, map_pos.y0 + 0.05, f"{right_letter})",
             fontsize=11*font_scale, ha='right', va='bottom', fontweight='bold')

    if show_colorbar:
        cax1.set_position([ax0_pos.x0, map_pos.y0 - 0.11, ax0_pos.width, 0.03])
        cax_map.set_position([map_pos.x0, map_pos.y0 - 0.11, map_pos.width, 0.03])

        sm = plt.cm.ScalarMappable(cmap=cmap_map, norm=norm_h)
        cb_map = fig.colorbar(sm, cax=cax_map, orientation='horizontal')
        cb_map.set_label('Local height [km]', fontsize=9*font_scale)
        cb_map.ax.tick_params(labelsize=8*font_scale)

    for ax in (ax0, ax0_twin):
        for orient_ in ['top', 'bottom', 'left', 'right']:
            ax.spines[orient_].set_linewidth(0.5)
        ax.tick_params(axis='both', labelsize=9*font_scale)

    if save:
        plt.savefig(Path(PLT_PATH, f"{title}.png"), dpi=400, bbox_inches='tight')
    plt.show()

In [ ]:
plot_integrated_results(
    results, times, sel, title=arc_of_interest,
    show_colorbar=True, left_letter='g', right_letter='h',
    save=False, font_scale=1.5, n_geodetics=20, label_every=3, legend_loc='center left',
)